# Praca domowa 3
Dane - Credit Cards Customers.

Problem - zbadaj którzy klienci są chętni do zrezygnowania z usługi.

Do zadania został użyty model Xgboost.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
plt.style.use('ggplot')
import dalex as dx
import pickle
import xgboost as xgb

from sklearn.model_selection import train_test_split
np.random.seed(23)

In [ ]:
input_df = pd.read_csv('data/preprocessed_dataset.csv')
input_df.info()

In [ ]:
y = input_df.loc[:,'Attrition']
X = input_df.drop('Attrition', axis='columns')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14) 

In [ ]:
xgboost = pickle.load(open("xgb_model.p", "rb" ))

In [ ]:
explainer = dx.Explainer(xgboost, X_train, y_train,  label = "Random Forest")

Attrition:

1-existing

0-attrited

# 1. Dla wybranej obserwacji ze zbioru danych wylicz predykcję modelu.

In [ ]:
explainer.predict(X_train)[0]

# 2. Dla wybranej obserwacji z punktu 1., wylicz dekompozycję predykcji modelu używając profili Ceteris Paribus/ ICE

In [ ]:
cp = explainer.predict_profile(X_train.iloc[0,:])

In [ ]:
cp.plot()

Predykcja dla obserwacji o indeksie 0 zmalałaby z 0.97 do 0.45 gdyby `Months_on_book`(liczba miesięcy od udzielenia pożyczki) wzrosło wzrosło do 44.
gdyby `Total_Trans_Amt`(liczba transakcji w ostatnich 12 miesiącach) spadło poniżej 1000 predykcja spdałaby do 0.18.

# 3.Wybierz dwie obserwacje ze zbioru danych, które mają różne profile CP

In [ ]:
cp = explainer.predict_profile(X_train.iloc[57,:])
cp.plot()

In [ ]:
cp = explainer.predict_profile(X_train.iloc[61,:])
cp.plot()

Porównujemy obserwację 57. i 61. dla których predykcje wynoszą 0.974 i 0.061. ZMienna `Gender` reprezentuje płeć użytkownika: 1-mężczyzna, 0-kobieta. Dla obserwacji 61. bycie kobietą zwiększa predykcję, a dla obserwacji 57. zmnijesza. Dla zmiennej `Months on book`. Dla obserwaji 57. predykcja rośnie wraz ze wzrtostem `Months on book`, z dwoma małymi spadkami dla wartości 15 oraz 30, Dla obserwacji 61. wzrost wartości zmiennej przyczynia się do spadku predykcji oraz występują podobne dwa spadki jak u obserwacji 57.
Dla zmiennej `Avg open to buy`(średni limit kredytu odnawialnego z ostatnich 12 miesięcy) obserwacja 57. ma niewielkie zmiany, a dla obserwacji 61. pojawia się duży spadek predykcji dla wartości 1380-3100.

# 4. Skomentuj poszczególne wyniki uzyskane w powyższych punktach.

W poprzedniej pracy domowej zastanawiało mnie dlaczego w zależnosci o ile wzrosło `Total trans amt` czasem dostawaliśmy przeciwnę wpływy na predykcję. Użycie profili Ceteris Paribus pozwala zobaczyć i zrozumieć skąd wynikają te różnice. Profil Ceteris Paribus pokazuje,które zmienne mają kluczowy wpływ na zmianę predykcji dla danej obserwacji i przy jakich wartosciach zmiany te są największe.